In [11]:
import pandas as pd
import csv


import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random as random


filePath  = "./AssetsValues/"

dataAssets = []
dataFramePerYear = []

firstYear = 2001
lastYear  = 2020

taxHavensList = {'AND': 'Andorra', 'AIA': 'Anguilla', 'ATG': 'Antigua and Barbuda', 'ABW':'Aruba', 'BHS': 'Bahamas', 'BHR':'Bahrain', 'BRB': 'Barbados', 'BLZ': 'Belize', 'BMU': 'Bermuda', 'VGB': 'Virgin Islands, British', 'CYM':'Cayman Islands', 'COK': 'Cook Islands',
				 'CRI': 'Costa Rica', 'CUW': 'Curaçao', 'CYP': 'Cyprus', 'DJI': 'Djibouti', 'DMA': 'Dominica', 'GIB': 'Gibraltar', 'GRD': 'Grenada', 'GGY': 'Guernsey', 'HKG': 'Hong Kong', 'IRL': 'Ireland', 'IMN': 'Isle of Man', 'JEY': 'Jersey', 'JOR': 'Jordan', 
				 'LBN': 'Lebanon', 'LBR': 'Liberia', 'LIE': 'Liechtenstein', 'LUX': 'Luxembourg', 'MAC': 'Macao', 'MDV': 'Maldives', 'MLT': 'Malta', 'MHL': 'Marshall Islands', 'MUS': 'Mauritius', 'FSM': 'Micronesia', 'MCO': 'Monaco', 'MSR': 'Montserrat', 'NRU': 'Nauru',
				 'ANT': 'Netherlands Antilles', 'NIU': 'Niue', 'PAN': 'Panama', 'WSM':'Samoa', 'SMR': 'San Marino', 'SYC': 'Seychelles', 'SGP': 'Singapore', 'KNA': 'St. Kitts and Nevis', 'LCA': 'St. Lucia', 'MAF':'St. Martin', 'VCT': 'St. Vincent and the Grenadines',
				 'CHE': 'Switzerland', 'TWN': 'Taiwan', 'TON': 'Tonga', 'TCA': 'Turks and Caicos', 'VUT': 'Vanuatu'}



In [12]:
def importCSVfiles():

	##########################################
	##
	## Importing Assets DataFrame
	##
	##########################################
	for i in range(firstYear, lastYear+1):
		dataFrame = pd.read_csv(filePath+"Assets"+str(i)+".csv", encoding='ISO-8859-1')
		dataFrame.columns.values[0] = "Country"

		dataFrame.fillna(0, inplace=True)                  ### QUITAR NANs
		dataFrame.replace(['-'], 0, inplace=True)          
		dataFrame.replace(['C'], 0, inplace=True)          ### "C" means that data are confidential. ## https://datahelp.imf.org/knowledgebase/articles/484340-what-does-the-symbol-c-mean-in-the-coordinated-d

		dataFrame.loc[:, dataFrame.columns != 'Country'] = dataFrame.loc[:, dataFrame.columns != 'Country'].applymap(lambda x: float(x.split()[0].replace(',', '')) if(hasattr(x, 'split')) else x)

		dataAssets.append(dataFrame)

In [16]:
print(len(dataAssets))

20


In [15]:
importCSVfiles()

for localYear in range(firstYear, lastYear+1):
	year = localYear

	countryList = list(dataAssets[year-firstYear].columns)
	countryList = countryList[1:len(countryList)]
	df = pd.DataFrame()

	#concatenating in one df per year
	for item in countryList:
		countryOwner = item
		df1 = pd.DataFrame()
		##########################################
		##
		## Each df1 is a Data Frame with the following columns
		## - column 1 (Target): Country that report asset over country of column 2
		## - column 2 (Source): Country that has to pay asset over country of column 1
		## - column 3 (Assets): Value of the Assets
		##
		## The graph would be something like:
		## country of column 1  <----(Asset)---- country of column 2
		##
		##########################################

		df1['Source'] = dataAssets[year-firstYear]['Country']
		df1['Assets'] = dataAssets[year-firstYear][countryOwner]
		df1.fillna(0.0, inplace=True)

		df1.insert(0, 'Target', countryOwner)
		df1 = df1[df1.Assets > 0]
		df1.reset_index(drop=True, inplace=True)

		df = pd.concat([df,df1], ignore_index=True)

	dataFramePerYear.append(df)

In [41]:
def takeAsset(c1, c2, year):
	if year < firstYear or year > lastYear:
		print("Year should be between (and included) "+str(firstYear)+" and "+str(lastYear))
		return
	
	df = dataFramePerYear[year-firstYear]
	dfaux = df[df['Target'] == c1].copy()
	dfauxaux = dfaux[dfaux['Source'] == c2].copy()

	if dfauxaux.empty:
		print("Theres no Asset value between "+str(c1)+" and "+str(c2)+" in "+str(year)+".")
	else:
		print(dfauxaux.head(1))

In [48]:
takeAsset('NLD', 'BEL', 2008)

     Target Source   Assets
3118    NLD    BEL  22356.0
